# Electron Study

The aim of this study is to determine whether the introduction of new track properties, namely `chi2rphi` and `chi2rz`, improve the performance of machine learning algorithms dedicated to telling whether electron-labeled track-trigger tracks are real or fake.

The Monte-Carlo samples available are a QCD sample, a Z boson to muon-muon sample, and a Z boson to electron-electron sample. Each sample is run for the D49 detector geometry and contains 1000 events, each of which has a pileup of about 200.

Much of the code here is run using the `ntupledicts` package, which can be found [here](https://github.com/cqpancoast/ntupledicts), along with a simple tutorial that covers all code used here.
The requirements for running this notebook are the same as the ones listed for running `ntupledicts` in the README.

In [1]:
from uproot import open as uproot_open
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Softmax
from tensorflow.keras.layers import Dense

from ntupledicts import operations as ndops
from ntupledicts import analyze as ndanl
from ntupledicts.operations import select as sel
from ntupledicts.ml import data as ndmldata
from ntupledicts.ml import predict as ndmlpred
from ntupledicts.ml import models as ndmlmodels

## Data Acquisition

Grab tracks from stored ntuples, perform cuts, process into datasets.
Take all electron tracks from the primary interaction and an equal number of fake tracks.

In [2]:
# List the ntuples we want data from
input_files = ["eventsets/ZMM_PU200_D49.root",
    "eventsets/ZEE_PU200_D49.root",
    "eventsets/QCD_PU200_D49.root"]
#input_files = ["eventsets/TTbar_PU200_D49_20.root"]

# Create list of uproot event sets for easy data access
event_sets = []
for input_file in input_files:
    event_sets.append(next(iter(uproot_open(input_file).values()))["eventTree"])
    
# What track properties do we want available to play with?
# We can select which ones we want our models to train on later
track_properties = ["pt", "eta", "z0", "nstub",
                        "hitpattern", "chi2", "bendchi2", "chi2rphi", "chi2rz"]
properties_by_track_type = {"trk": track_properties + ["matchtp_pdgid", "genuine"],
                            "matchtrk": track_properties, 
                            "tp": ["pdgid", "eventid"]}

# Create ntuple dict from event sets
ntuple_dict = ndops.uproot_ntuples_to_ntuple_dict(event_sets, properties_by_track_type)

### Apply Cuts

We want electron matchtrks from the primary interaction and an equal number of fake trks.

In [ ]:
# Would we like to consider only a portion of this dataset? (Typically done for speed reasons.)
reduce_ntuple_dict = False
reduction_size = 1000000  # number of tracks to reduce to
if reduce_ntuple_dict:
    ntuple_dict = ndops.reduce_ntuple_dict(ntuple_dict, reduction_size)

# Get electron matchtrks from the primary interaction, fake trks
# Note that cutting on tp's also cuts on matchtrks
el_sel = sel([sel(11), sel(-11)])
ntuple_dict = ndops.cut_ntuple_dict(ntuple_dict, {"tp": {"pdgid": el_sel, "eventid": sel(0)},
                                            "matchtrk": {"hitpattern": sel([sel(-999)], invert=True)},
                                                 "trk": {"genuine": sel(0)}})
matchtrk_els = ntuple_dict["matchtrk"]
matchtrk_els["genuine"] = [1 for _ in range(ndops.track_prop_dict_length(matchtrk_els))]
    # All matchtrks are genuine
trk_not_gens = ntuple_dict["trk"]
trk_not_gens = ndops.reduce_track_prop_dict(trk_not_gens, ndops.track_prop_dict_length(matchtrk_els))

# Cut down the number of not genuine tracks to the number of electrons
print("Number of primary interaction electron tracks: {}".format(ndops.track_prop_dict_length(matchtrk_els)))
print("Number of not genuine tracks: {}".format(ndops.track_prop_dict_length(trk_not_gens)))

# Now, add them together.
# A UserWarning is expected here, as trk_not_gens has a property "matchtp_pdgid" that matchtrk_els doesn't have.
els_and_fakes_tpd = ndops.shuffle_track_prop_dict(
                        ndops.add_track_prop_dicts([matchtrk_els, trk_not_gens]),
                        seed=42)

### Add Stub-Based Track Properties

Use hitpattern and eta to determine how many missing 2S and PS stubs there are for each track. (Note that this is post-cuts — if we want to place cuts on these, we'll move this cell before the cut application cell.)

In [ ]:
# Count layers meeting these conditions for each track. (see ntupledicts.analyze for documentation)
missing_2S_layer = lambda expected, hit, ps: not ps and expected and not hit
missing_PS_layer = lambda expected, hit, ps: ps and expected and not hit

els_and_fakes_tpd["missing2S"] = ndanl.create_stub_info_list(els_and_fakes_tpd,
        ndanl.basic_process_stub_info(missing_2S_layer))
els_and_fakes_tpd["missingPS"] = ndanl.create_stub_info_list(els_and_fakes_tpd,
        ndanl.basic_process_stub_info(missing_PS_layer))

### Process into Datasets

Process the ntuple dict above into `TrackPropertiesDataset`s.

In [ ]:
common_track_props = ["pt", "eta", "z0", "bendchi2", "nstub", "missing2S", "missingPS"]  
    # Track properties that both sets of models will use.
non_sc2_data_props = common_track_props + ["chi2"]  # Our first set of models will also include chi2.
label_property = "genuine"  # What property are we trying to predict?
split_list = [.7, .2, .1]   # How many datasets should we create, and with what relative sizes?
train_ds, eval_ds, test_ds = ndmldata.TrackPropertiesDataset(els_and_fakes_tpd,
                                                             label_property,
                                                             non_sc2_data_props).split(split_list)

## Making Models

Build two neural networks and two gradient boosted decision trees, train one of them on data containing the split chi2 variables and one of them on data without them. Also define two sets of predictive cuts to compare our models against.

### Chi2 Model Training

In [ ]:
# Make our non-split chi2 set of models
NN = ndmlmodels.make_neuralnet(train_ds, eval_dataset=eval_ds, hidden_layers=[15, 8], epochs=10)
GBDT = ndmlmodels.make_gbdt(train_ds)

### Split Chi2 Model Training

In [ ]:
# Remove chi2 and introduce the split chi2's as active data properties
sc2_data_props = common_track_props + ["chi2rphi", "chi2rz"]
for ds in [train_ds, eval_ds, test_ds]:
    ds.set_active_data_properties(sc2_data_props)

# Make our split chi2 set of models (sc2 stands for split chi2)
NN_sc2 = ndmlmodels.make_neuralnet(train_ds, eval_dataset=eval_ds, hidden_layers=[15, 8], epochs=10)
GBDT_sc2 = ndmlmodels.make_gbdt(train_ds)

### Predictive Cuts

In [ ]:
# Define predictive cuts
chi2_cuts = {"chi2": sel(0, 70), "bendchi2": sel(0, 2.5)}
sc2_cuts = {"chi2rz": sel(0, 5), "bendchi2": sel(0, 2.5)}  # Chi2rz cuts perform much better for electrons than chi2rphi

### Model Predictions

Use the test dataset `test_ds` that hasn't been used for training to make predicted labels. These will be probablistic in the case of the models `NN` and `GBDT` and exact in the case of `cuts`. Store these predictions in `test_ds` for easy future access.

In [ ]:
# Add normal chi2 predictions
test_ds.add_prediction("NN", ndmlpred.predict_labels(NN, test_ds.get_data(non_sc2_data_props)))
test_ds.add_prediction("GBDT", ndmlpred.predict_labels(GBDT, test_ds.get_data(non_sc2_data_props)))
test_ds.add_prediction("chi2_cuts", ndmlpred.predict_labels_cuts(chi2_cuts, test_ds))

# Add split chi2 predictions
test_ds.add_prediction("NN_sc2", ndmlpred.predict_labels(NN_sc2, test_ds.get_data(sc2_data_props)))
test_ds.add_prediction("GBDT_sc2", ndmlpred.predict_labels(GBDT_sc2, test_ds.get_data(sc2_data_props)))
test_ds.add_prediction("sc2_cuts", ndmlpred.predict_labels_cuts(sc2_cuts, test_ds))

## Plotting Model Predictions

Now we'll compare the two sets of predictions to see which ones perform better. These plots will measure the true positive rate and false positive rate (TPR and FPR) of these predictions with respect to various threshold values and track properties.

In [ ]:
from ntupledicts import plot as ndplot
from ntupledicts.ml import plot as ndmlplot
from matplotlib.pyplot import cla, sca, gca, show

(These import statementa are down here for debug purposes - so we don't have to rerun all the cells to change something in `ml.plot` suite or add some new `matplotlib` import.)

### ROC Curve

Plot the models' paths through TPR/FPR space as the threshold on probablistically predicted labels changes. Also includes the cuts' point in that space.

In [ ]:
xlims=(0, 1)
ylims=(0, 1)

ndmlplot.plot_rocs(test_ds, ["NN", "GBDT"], ["chi2_cuts"], xlims=xlims, ylims=ylims)
ndmlplot.plot_rocs(test_ds, ["NN_sc2", "GBDT_sc2"], ["sc2_cuts"], xlims=xlims, ylims=ylims)

### Particle TPR and FPR by Threshold

Plot TPR and FPR for particles of a particular type as the threshold for assigning probablistic values to either genuine or not genuine shifts from zero to one. Make plots for both sets of models to compare performance.

In [ ]:
# TPR/FPR vs decision thresh * models, ov. particle type
for pred_comparison, pred_comp_name in zip(
        [ndmlpred.true_positive_rate, ndmlpred.false_positive_rate],
        ["TPR", "FPR"]):
    ax = gca()
    for pred_name in test_ds.get_all_prediction_names():
        if "cut" in pred_name:
            continue
        ax = ndmlplot.plot_pred_comparison_by_threshold(test_ds,
                pred_name, pred_comparison, legend_id=pred_name, ax=ax)
    ax.set_ylabel(pred_comp_name)
    ax.set_title("{} of model predictions by threshold".format(pred_comp_name))
    ax.legend()
    show()
    cla()

### Particle TPR and FPR by Track Property

Plot TPR and FPR for particles of a particular type for binned values of some track property. The aim of this is to get a sense of how model performance depends upon track properties like pT and eta. Make plots for both sets of models to compare performance.

In [ ]:
# TPR/FPR vs some track prop * models, ov. particle type.
# Parametrized by binning property.

# A dictionary from track properties to bin specifiers
binning_dict = {"pt": (2, 100, 30)}

for binning_prop, bins in zip(binning_dict.keys(), binning_dict.values()):
    for pred_comparison, pred_comp_name in zip(
            [ndmlpred.true_positive_rate, ndmlpred.false_positive_rate],
            ["TPR", "FPR"]):
        ax = gca()
        for pred_name in test_ds.get_all_prediction_names():
            ax = ndmlplot.plot_pred_comparison_by_track_property(
                    test_ds, pred_name,
                    pred_comparison, binning_prop, bins=bins,
                    legend_id=pred_name, ax=ax)
        ax.set_ylabel(pred_comp_name)
        ax.set_title("{} of model predictions vs. {}".format(pred_comp_name, binning_prop))
        ax.legend()
        show()
        cla()

## Takeaways

Bacon ipsum dolor amet...